# Author
[Koen Aerts](https://koenaerts.ca/) @ [Mobia Technology Innovations](https://mobia.io)

[myOpenHealth](https://mobia.io/healthcare/)

# Intro
In this notebook we will use the CSV files created from the "04_prep_imported_data" notebook to train, validate and test our model.

Many other models that have been published seem to be using a Convolutional Neural Network build on Keras layers. Instead, we will be using Tensorflow's DNNClassifier Estimator, which results in code that is much shorter and easier to read, and leaves all the complexities of building the Neural Network to Tensorflow.

# Initialize

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Load Data
Load the CSV files with training, validation, and testing data. These files are generated by the "04_prep_imported_data" notebook.

In [ ]:
# Training dataset.
df = pd.read_csv("train.csv", header=None)
x_train = df.values[:, :-1]
y_train = df.values[:, -1].astype(int)

# Validation dataset.
df = pd.read_csv("validate.csv", header=None)
x_validate = df.values[:, :-1]
y_validate = df.values[:, -1].astype(int)

# Test dataset.
df = pd.read_csv("test.csv", header=None)
x_test = df.values[:, :-1]
y_test = df.values[:, -1].astype(int)

In [ ]:
del df

# Visualize Data
Display one Normal, and one Abnormal heartbeat.

In [ ]:
C0 = np.argwhere(y_train == 0).flatten()
C1 = np.argwhere(y_train == 1).flatten()

In [ ]:
x = np.arange(0, 187)*8/1000.0

plt.figure(figsize=(20,12))
plt.plot(x, x_train[C0, :][0], label="Normal") # Display first normal beat.
plt.plot(x, x_train[C1, :][0], label="Abnormal") # Display first abnormal beat.
plt.legend()
plt.title("1-beat ECG for every category", fontsize=20)
plt.ylabel("Normalized Amplitude (0 - 1)", fontsize=15)
plt.xlabel("Time (ms)", fontsize=15)
plt.show()

# Train Model
Use the Tensorflow Estimator API to build out the model.

In [ ]:
feature_columns = [tf.feature_column.numeric_column('beat', shape=[187])]

estimator = tf.estimator.DNNClassifier(
   feature_columns=feature_columns,
   hidden_units=[256, 64, 16],
   optimizer=tf.train.AdamOptimizer(1e-4),
   n_classes=2,
   dropout=0.1,
   model_dir='ecg_model'
)

input_fn_train = tf.estimator.inputs.numpy_input_fn(
    x={'beat': x_train},
    y=y_train,
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

In [ ]:
estimator.train(input_fn=input_fn_train, steps=400000)

# Validate Model
Evaluate how well the model is working.

In [ ]:
input_fn_validate = tf.estimator.inputs.numpy_input_fn(
    x={'beat': x_validate},
    y=y_validate,
    num_epochs=1,
    shuffle=False
)

In [ ]:
accuracy_score = estimator.evaluate(input_fn=input_fn_validate)
print('\nTest Accuracy: {0:f}%\n'.format(accuracy_score['accuracy']*100))

# Test Model
Testing the model by doing predictions.

In [ ]:
input_fn_test = tf.estimator.inputs.numpy_input_fn(
 x={'beat': x_test},
 y=y_test,
 num_epochs=1,
 shuffle=False
)

In [ ]:
predictions = estimator.predict(input_fn=input_fn_test)

In [ ]:
totvals = 0
totwrong = 0

for prediction, expected in zip(predictions, y_test):
    totvals = totvals + 1
    catpred = prediction['class_ids'][0]
    certainty = prediction['probabilities'][catpred] * 100
    if (expected != catpred):
        totwrong = totwrong + 1
        #print (prediction)
        print('Real: ', expected, ', pred: ', catpred, ', cert: ', certainty)

print('Accuracy: ', ((totvals - totwrong) * 100.0 / totvals))
print('Wrong: ', totwrong, ' out of ', totvals)

# Monitoring
Use TensorBoard in a separate tab in your browser to analyze your model.

In [ ]:
# Monitoring with TensorBoard
from google.datalab.ml import TensorBoard
TensorBoard().start('ecg_model')
TensorBoard().list()

In [ ]:
# Stop TensorBoard
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid ', pid)

# Export Model
Exporting the model to make predictions.

In [ ]:
# Build receiver function, and export.
#feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)
#serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
feature_placeholders = { 'beat': tf.placeholder(dtype=tf.float32, shape=(187,)) }
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(feature_placeholders)
export_dir = estimator.export_savedmodel('ecg_serving', serving_input_receiver_fn, strip_default_attrs=True)